In [ ]:
import psycopg2
import calendar
import datetime
import time
import requests
import schedule

In [ ]:
#подключение к базе
conn = psycopg2.connect( host="",
       database="",
       user="",
       password="",
       port="")
# запрос к базе, текст запроса хранится отдельно
def get_query(query):
    try:
        cur = conn.cursor() cur.execute(query)
        rows = cur.fetchall()
        return rows
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

Запросы с метриками

In [ ]:
queries = {'Yesterday sales': """
                            SELECT date, daily_sum, monthly_sum
                            FROM (
                            SELECT 
                            DISTINCT(transaction_created_at::date) AS date,
                            SUM(amount::float) OVER (PARTITION BY transaction_created_at::date) AS daily_sum
                            SUM(amount::float) OVER (PARTITION BY EXTRACT (month FROM transaction_created_at::date)) AS montly_sum
                            FROM public.payments
                            ORDER BY date) AS sales
                            WHERE date = current_date - INTEGER '1'
                            """,
           'Yesterday events': """
                            SELECT 
                            happened_at::date AS date,
                            public.events_dict._description AS event,
                            COUNT(DISTINCT(_user_id)) AS users
                            FROM public.events
                             LEFT JOIN public.events_dict ON id=event.id
                            WHERE happened_at::date = current_date - INTEGER '1'
                            GROUP BY happened_at::date, _description
                            """,
           'Yesterday unique events': """
                            SELECT 
                            events.happened_at::date AS date,
                            descriptions._description AS event,
                            COUNT(DISTINCT(events._user_id)) AS count
                            FROM public.events AS events
                             INNER JOIN (
                              SELECT happened_at::date AS date, _user_id, event_id
                              FROM public.events
                              WHERE happened_at::date = current_date - INTEGER '1 AND event_id::int = 3') users
                              ON events._user_id = users._user_id
                             LEFT JOIN public.events_dict AS descriptions ON descriptions.id = events.event_id
                            WHERE happened_at::date = current_date - INTEGER '1'
                            GROUP BY events.happened_at::date, descriptions._description
                            """
}

Метрики продаж

In [ ]:
def get_sales_metric():
    today = datetime.datetime.now()
    sales = get_query(queries['Yesterday sales'])
    date = sales[0][0]
    yesterday_sales = sales[0][1]
    this_month_sales = sales[0][2]
    days_passed = sales[0][0].day
    days_ttl = calendar.monthrange(today.year, today.month)[1]
    runrate = this_month_sales / days_passed * days_ttl
    return date, int(yesterday_sales), int(this_month_sales), int(runrate)

Метрики событий за вчера

In [ ]:
def get_events_metric(): 
    events_rows = get_query(queries['Yesterday events'] 
    events = {}
    for event in events_rows:
        events[event[1]] = event[2]
    call_1_success = events['Ученик ответил на звонок оператора 1Л']
    intro_lesson = events["Назначение ВУ"]
    return call_1_success, intro_lesson

Уникальные события

In [ ]:
def get_unique_events_metric(): 
    events_rows = get_query(queries['Yesterday unique events'] 
    events = {}
    for event in events_rows:
        events[event[1]] = event[2]
    yesterday_requests = events['Назначение задачи на звонок 1Л']
    yesterday_success_call_1 = events['Учение ответил на звонок оператора 1л']
    yesterday_lesson = events['Назначение ВУ']
    to_call_conv = int(yesterday_success_call_1 / yesterday_requests * 100) 
    to_lesson_conv = int(yesterday_lesson / yesterday_success_call_1 * 100) 
    return yesterday_requests, yesterday_success_call_1, yesterday_lesson, to_call_conv, to_lesson_conv

Телеграм бот для отправки уведомления

In [ ]:
def telegram_bot_sendtext(bot_message):
    bot_token = ''
    bot_chatID = ''
    send_text = 'https://api.telegram.org/bot' + bot_token + 
                '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

Сам отчет

In [ ]:
def report():
    metric_1 = get_salec_metric()
    metric_2 = get_unique_events_metric()
    metric_3 = get_events_metric()
    list = ['Привет! Продажи на {}: {} руб. \n'.format(metric_1[0], metric_1[1]),
            'За месяц накоплено: {} руб., прогноз на конец мес.: {} руб. \n'.format(metric_1[2], metric_1[3]),
            'Мы получили {} заявок на звонки, {} прозвонили ({}%) \n'.format(metric_2[0], metric_2[1], metric_2[3]),
            '{} из этих звонков закончились назначением урока ({}%) \n'.format(metric_2[2], metric_2[4]),
            'В общем сложности мы сделали {} звонков и назначили {} уроков \n'.format(metric_3[0], metric_3[1])
            ]
    report = ''.join(list)
    telegram_bot_sendtext(report)

Запуск

In [ ]:
# таймер на 9 утра
schedule.every().day.at('09:00').do(report)
while True: 
    schedule.run_pending() 
    time.sleep(1)
# закрываем соединение к БД
if conn is not None: conn.close()